In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from sklearn.model_selection import train_test_split

np.random.seed(42)
tf.random.set_seed(42)


def read_data(files):
    data_matrix = []
    for filename in files:
        single_sensor_matrix = pd.read_csv(filename, delim_whitespace=True, header=None).values
        data_matrix.append(single_sensor_matrix)
    data_matrix = np.array(data_matrix)
    data_matrix = np.transpose(data_matrix, (1, 2, 0))
    return data_matrix


train_data_folder = r'UCI HAR Dataset\train\Inertial Signals'
test_data_folder = r'UCI HAR Dataset\test\Inertial Signals'

sensor_names = [filename.replace('_train.txt', '').replace('_test.txt', '') for filename in os.listdir(train_data_folder)]
train_files = [os.path.join(train_data_folder, filename + '_train.txt') for filename in sensor_names]
test_files = [os.path.join(test_data_folder, filename + '_test.txt') for filename in sensor_names]

X_train = read_data(train_files)
X_test = read_data(test_files)


def get_labels(split):
    df = pd.read_csv(f'UCI HAR Dataset\\{split}\y_{split}.txt', delim_whitespace=True, header=None)
    return pd.get_dummies(df[0])


y_train = get_labels('train')
y_test = get_labels('test')

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, stratify=y_train, random_state=42)


def create_model(input_dim, timesteps, n_hidden, n_classes):
    model = Sequential()
    model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
    model.add(Dropout(0.5))
    model.add(Dense(n_classes, activation='softmax'))
    return model


input_dim = X_train.shape[2]
timesteps = X_train.shape[1]
n_classes = y_train.shape[1]
n_hidden = 16

model = create_model(input_dim, timesteps, n_hidden, n_classes)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 16)                1664      
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense (Dense)               (None, 6)                 102       
                                                                 
Total params: 1,766
Trainable params: 1,766
Non-trainable params: 0
_________________________________________________________________


In [2]:
# Define callbacks
epochs = 50
batch_size = 32
early_stopping_cb = EarlyStopping(patience=5, restore_best_weights=True)
checkpoint_cb = ModelCheckpoint("activity_detection_model.h5", save_best_only=True)
tensorboard_cb = TensorBoard(log_dir="logs")

history = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=batch_size, epochs=epochs,
                    callbacks=[early_stopping_cb, checkpoint_cb, tensorboard_cb])

# Evaluate on test set
test_loss, test_acc = model.evaluate(X_test, y_test)

print("Test accuracy:", test_acc)

Epoch 1/50
161/161 [==============================] - 12s 57ms/step - loss: 1.4538 - accuracy: 0.4190 - val_loss: 1.1716 - val_accuracy: 0.5345
Epoch 2/50
161/161 [==============================] - 8s 52ms/step - loss: 1.0581 - accuracy: 0.5754 - val_loss: 0.8861 - val_accuracy: 0.6301
Epoch 3/50
161/161 [==============================] - 9s 55ms/step - loss: 0.8978 - accuracy: 0.6172 - val_loss: 0.7652 - val_accuracy: 0.6364
Epoch 4/50
161/161 [==============================] - 9s 54ms/step - loss: 0.8229 - accuracy: 0.6236 - val_loss: 0.7080 - val_accuracy: 0.6741
Epoch 5/50
161/161 [==============================] - 9s 55ms/step - loss: 0.7902 - accuracy: 0.6263 - val_loss: 0.7008 - val_accuracy: 0.6700
Epoch 6/50
161/161 [==============================] - 9s 54ms/step - loss: 0.7824 - accuracy: 0.6277 - val_loss: 0.6791 - val_accuracy: 0.6677
Epoch 7/50
161/161 [==============================] - 9s 55ms/step - loss: 0.7521 - accuracy: 0.6502 - val_loss: 0.6561 - val_accuracy: 0.688

In [3]:
# View in TensorBoard
!tensorboard --logdir=logs

^C
